In [1]:
import os
os.environ["LOKY_MAX_CPU_COUNT"]="6"
import pickle
import pandas as pd
import gc
import numpy as np
import psutil
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, TargetEncoder
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from  lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
import joblib
import datetime
import random

In [30]:
gc.collect()

2591

In [29]:
round(psutil.virtual_memory().available/(1024**3),2)

16.06

In [19]:
df = pd.read_parquet("ga_sessions.parquet")
df.shape

(1860042, 18)

In [20]:
df.head(5)

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
0,9055434745589932991.1637753792.1637753792,2108382700.1637753791,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,None,360x720,Chrome,Russia,Zlatoust
1,905544597018549464.1636867290.1636867290,210838531.1636867288,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,None,385x854,Samsung Internet,Russia,Moscow
2,9055446045651783499.1640648526.1640648526,2108385331.1640648523,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,None,360x720,Chrome,Russia,Krasnoyarsk
3,9055447046360770272.1622255328.1622255328,2108385564.1622255328,2021-05-29,05:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,None,NOBKLgtuvqYWkXQHeYWM,None,mobile,None,Xiaomi,None,393x786,Chrome,Russia,Moscow
4,9055447046360770272.1622255345.1622255345,2108385564.1622255328,2021-05-29,05:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,None,None,None,mobile,None,Xiaomi,None,393x786,Chrome,Russia,Moscow


In [31]:
df5 = df.loc[np.random.randint(0,1860043),:].to_json()

In [33]:
print(df5)

{"session_id":"2657526989008451789.1625077950.1625077950","client_id":"618753719.1625077965","visit_date":1625011200000,"visit_time":"21:00:00","visit_number":2,"utm_source":"kjsLglQLzykiRbcDiGcD","utm_medium":"cpc","utm_campaign":null,"utm_adcontent":null,"utm_keyword":null,"device_category":"mobile","device_os":null,"device_brand":"Samsung","device_model":null,"device_screen_resolution":"360x800","device_browser":"Chrome","geo_country":"Russia","geo_city":"Saint Petersburg"}


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1860042 entries, 0 to 1860041
Data columns (total 18 columns):
 #   Column                    Dtype 
---  ------                    ----- 
 0   session_id                object
 1   client_id                 object
 2   visit_date                object
 3   visit_time                object
 4   visit_number              int64 
 5   utm_source                object
 6   utm_medium                object
 7   utm_campaign              object
 8   utm_adcontent             object
 9   utm_keyword               object
 10  device_category           object
 11  device_os                 object
 12  device_brand              object
 13  device_model              object
 14  device_screen_resolution  object
 15  device_browser            object
 16  geo_country               object
 17  geo_city                  object
dtypes: int64(1), object(17)
memory usage: 255.4+ MB


In [28]:
df["visit_number"].dtypes.kind

'i'

In [29]:
d = {}
d_changes = {'O': str, "i" : int}

for column in df.columns.tolist():
    d[column] = d_changes[df[column].dtype.kind]

print(d)

{'session_id': <class 'str'>, 'client_id': <class 'str'>, 'visit_date': <class 'str'>, 'visit_time': <class 'str'>, 'visit_number': <class 'int'>, 'utm_source': <class 'str'>, 'utm_medium': <class 'str'>, 'utm_campaign': <class 'str'>, 'utm_adcontent': <class 'str'>, 'utm_keyword': <class 'str'>, 'device_category': <class 'str'>, 'device_os': <class 'str'>, 'device_brand': <class 'str'>, 'device_model': <class 'str'>, 'device_screen_resolution': <class 'str'>, 'device_browser': <class 'str'>, 'geo_country': <class 'str'>, 'geo_city': <class 'str'>}


In [31]:
d["visit_date"]

str

In [7]:
df[df.duplicated()]
#нет дубликатов

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city


In [8]:
df2 = pd.read_parquet("ga_hits.parquet")
df2.head()

,session_id,hit_date,hit_time,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,event_label,event_value
0,5639623078712724064.1640254056.1640254056,2021-12-23,597864.0,30,event,None,sberauto.com/cars?utm_source_initial=google&ut...,quiz,quiz_show,None,NaN
1,7750352294969115059.1640271109.1640271109,2021-12-23,597331.0,41,event,None,sberauto.com/cars/fiat?city=1&city=18&rental_c...,quiz,quiz_show,None,NaN
2,885342191847998240.1640235807.1640235807,2021-12-23,796252.0,49,event,None,sberauto.com/cars/all/volkswagen/polo/e994838f...,quiz,quiz_show,None,NaN
3,142526202120934167.1640211014.1640211014,2021-12-23,934292.0,46,event,None,sberauto.com/cars?utm_source_initial=yandex&ut...,quiz,quiz_show,None,NaN
4,3450086108837475701.1640265078.1640265078,2021-12-23,768741.0,79,event,None,sberauto.com/cars/all/mercedes-benz/cla-klasse...,quiz,quiz_show,None,NaN


In [9]:
df2[df2.duplicated()]
#нет дубликатов

,session_id,hit_date,hit_time,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,event_label,event_value


In [10]:
df_client = df.copy()
df_event = df2.copy()

In [11]:
df_client.shape, df_event.shape

((1860042, 18), (15726470, 11))

In [12]:
df_client.loc[df["visit_number"]>2, "session_id"]

13         9055469620715506713.1635878177.1635878177
14         9055469620715506713.1636559415.1636559415
23         9055497958914887309.1634048072.1634048072
24         9055497958914887309.1634223619.1634223619
25         9055497958914887309.1634224644.1634224644
                             ...                    
1859997    9055237430499220580.1639607417.1639607417
1860002    9055248417020050011.1634075541.1634075541
1860003    9055248417020050011.1634078104.1634078104
1860004    9055248417020050011.1634078414.1634078414
1860020    9055329372850890545.1629287172.1629287172
Name: session_id, Length: 248037, dtype: object

In [13]:
df_client["session_id_ch"] = df_client["session_id"].str.split(".").str[0]
df_event["session_id_ch"] = df_event["session_id"].str.split(".").str[0]

In [12]:
df_client["session_id_ch"].nunique()

1391719

In [13]:
df_event["session_id_ch"].nunique()

1320814

In [15]:
target_list = ['sub_car_claim_click', 'sub_car_claim_submit_click',
'sub_open_dialog_click', 'sub_custom_question_submit_click',
'sub_call_number_click', 'sub_callback_submit_click', 'sub_submit_success',
'sub_car_request_submit_click']

In [16]:
df_event["target"] = df_event["event_action"].apply(lambda x : 1 if x in target_list else 0)

In [17]:
df_event_total = df_event.groupby("session_id_ch")["target"].sum().reset_index()

In [19]:
df_event_total.shape

(1320814, 2)

In [20]:
l = df_client.columns.tolist()

for i in l:
    print(i, df_client[i].nunique())

session_id 1860042
client_id 1391719
visit_date 226
visit_time 85318
visit_number 537
utm_source 293
utm_medium 56
utm_campaign 412
utm_adcontent 286
utm_keyword 1219
device_category 3
device_os 13
device_brand 206
device_model 104
device_screen_resolution 5039
device_browser 57
geo_country 166
geo_city 2548
session_id_ch 1391719


client_id 1391719=session_id_ch 1391719, количество клиентов равно количеству session_id_ch ,вероятнее всего 1 клиенту соответсвует 1 запись session_id_ch

CR (Conversion Rate) — показатель конверсии из визита (уникальный
session_id) в любое целевое действие в рамках одного визита (в случае
наличия >1 целевого действия — считать все как одно).

In [22]:
df_event_total["target"].value_counts()

target
0     1276903
1       22627
3        9062
2        5924
4        2655
       ...   
40          1
61          1
74          1
96          1
87          1
Name: count, Length: 73, dtype: int64

In [23]:
df_event_total["target"]=df_event_total["target"].apply(lambda x: 0 if x==0 else 1)

In [24]:
df_event_total["target"].value_counts(normalize = True)

target
0    0.966755
1    0.033245
Name: proportion, dtype: float64

распределение целевой переменной очень сильно несбалансировано

In [25]:
df_client.columns

Index(['session_id', 'client_id', 'visit_date', 'visit_time', 'visit_number',
       'utm_source', 'utm_medium', 'utm_campaign', 'utm_adcontent',
       'utm_keyword', 'device_category', 'device_os', 'device_brand',
       'device_model', 'device_screen_resolution', 'device_browser',
       'geo_country', 'geo_city', 'session_id_ch'],
      dtype='object')

In [26]:
df_client.isna().sum()

session_id                        0
client_id                         0
visit_date                        0
visit_time                        0
visit_number                      0
utm_source                       97
utm_medium                        0
utm_campaign                 219603
utm_adcontent                335615
utm_keyword                 1082061
device_category                   0
device_os                   1070138
device_brand                 118678
device_model                1843704
device_screen_resolution          0
device_browser                    0
geo_country                       0
geo_city                          0
session_id_ch                     0
dtype: int64

In [40]:
df_client_total = df_client.groupby("session_id_ch")[['utm_medium',
        'device_category',  'device_brand','device_browser',
       'geo_country', 'geo_city', 'utm_campaign', "utm_source" , 'utm_adcontent' , "device_os"]].last().reset_index()

In [41]:
df_client_total.shape

(1391719, 11)

In [42]:
df_client_total.isna().sum()

session_id_ch           0
utm_medium              0
device_category         0
device_brand        78927
device_browser          0
geo_country             0
geo_city                0
utm_campaign        96645
utm_source              6
utm_adcontent      180972
device_os          760552
dtype: int64

In [ ]:
df_total = df__client_total.merge(df_event_total, on = "session_id_ch", how = "inner")

In [ ]:
df_total.to_parquet("client_event_total.parquet")

In [2]:
df_total1 = pd.read_parquet("client_event_total.parquet")
df_total1.head()

,session_id_ch,utm_medium,device_category,device_brand,device_browser,geo_country,geo_city,utm_campaign,utm_source,utm_adcontent,device_os,target
0,1000009318903347362,cpm,mobile,Samsung,Chrome,Russia,Gelendzhik,FTjNLDyTrXaWYgZymFkV,MvfHsxITijuriZxsqZqt,PkybGvWbaqORmxjNunqZ,None,0
1,1000010177899156286,(none),mobile,Samsung,Samsung Internet,Russia,Voronezh,LTuZkdKfxRGVceoWkVyg,fDLlAcSmythWSCVMvqvL,JNHcPlZPxEMWDnRiyoBf,Android,0
2,1000013386240115915,banner,mobile,Samsung,Chrome,Russia,Cherkessk,gecBYcKZCPMcVYdSSzKP,ZpYIoDJMcFzVoPFsHGJL,JNHcPlZPxEMWDnRiyoBf,Android,0
3,1000017303238376207,banner,mobile,Realme,Chrome,Russia,Irkutsk,LEoPHuyFvzoNfnzGgfcd,ZpYIoDJMcFzVoPFsHGJL,JNHcPlZPxEMWDnRiyoBf,None,0
4,1000020580299877109,(none),mobile,Apple,Safari,Russia,Moscow,LTuZkdKfxRGVceoWkVyg,fDLlAcSmythWSCVMvqvL,JNHcPlZPxEMWDnRiyoBf,None,0


In [3]:
df_total = df_total1.copy()

In [4]:
df_total["device_os"].value_counts()

device_os
Android          365529
iOS              142280
Windows           57435
Macintosh         15253
Linux              3732
(not set)           308
Chrome OS            60
BlackBerry           16
Tizen                 6
Firefox OS            2
Nokia                 2
Samsung               2
Windows Phone         2
Name: count, dtype: int64

In [5]:
df_total.isna().sum()

session_id_ch           0
utm_medium              0
device_category         0
device_brand        73886
device_browser          0
geo_country             0
geo_city                0
utm_campaign        89233
utm_source              5
utm_adcontent      168669
device_os          736083
target                  0
dtype: int64

In [6]:
df_total["device_browser"].unique()

array(['Chrome', 'Samsung Internet', 'Safari', 'YaBrowser', 'Edge',
       'Android Webview', 'Safari (in-app)', 'Firefox', 'Opera',
       'UC Browser', 'Opera Mini', 'Instagram 209.0.0.21.119 Android',
       'Mozilla Compatible Agent', 'Instagram 208.0.0.32.135 Android',
       'Instagram 202.0.0.23.119', '[FBAN', 'Android Runtime',
       'Instagram 213.0.0.29.120 Android', 'Puffin', 'Android',
       'com.vk.vkclient', 'Android Browser', 'Maxthon',
       'Internet Explorer', 'Instagram 207.0.0.28.118', 'MRCHROME',
       'helloworld', 'Instagram 199.1.0.34.119 Android',
       'Instagram 192.0.0.37.119', 'Amazon Silk', 'SeaMonkey',
       'Instagram 194.0.0.36.172 Android', 'Instagram 216.0.0.12.135',
       'Instagram 202.0.0.37.123 Android', 'MyApp', '(not set)',
       'Coc Coc', 'NetFront', 'Instagram 201.0.0.23.111',
       'Instagram 203.0.0.26.117', 'Mozilla',
       'Instagram 192.0.0.35.123 Android',
       'Instagram 212.0.0.38.119 Android', 'Instagram 208.0.0.26.131',


In [7]:
df_total["device_browser"].value_counts()

device_browser
Chrome                              735917
Safari                              313986
YaBrowser                            88304
Safari (in-app)                      59115
Android Webview                      45546
Samsung Internet                     34918
Opera                                18690
Firefox                              11460
Edge                                 11016
UC Browser                             584
Mozilla Compatible Agent               321
Android Runtime                        238
[FBAN                                  112
Instagram 208.0.0.32.135 Android        70
MRCHROME                                62
Instagram 209.0.0.21.119 Android        49
Android Browser                         46
Puffin                                  34
Internet Explorer                       24
Opera Mini                              22
Android                                 19
Instagram 202.0.0.23.119                17
Maxthon                                

In [8]:
df_total["device_brand"].unique()

array(['Samsung', 'Realme', 'Apple', 'Huawei', 'Xiaomi', None, 'Vivo', '',
       'OPPO', '(not set)', 'OnePlus', 'Sony', 'BQ', 'ZTE', 'Asus',
       'Umidigi', 'Meizu', 'itel', 'LG', 'LeEco', 'Nokia', 'Blackview',
       'Google', 'Lenovo', 'BlackBerry', 'Tecno', 'Cubot', 'Alcatel',
       'Neffos', 'Vsmart', 'Motorola', 'Infinix', 'TCL', 'Vertex',
       'Philips', 'Oukitel', 'Vernee', 'Micromax', 'HOMTOM', 'CAT',
       'Prestigio', 'Ulefone', 'DOOGEE', 'Leagoo', 'TP-Link',
       'SonyEricsson', 'HTC', 'Mozilla', 'Karbonn', 'Sharp', 'Wileyfox',
       'POCO', 'A1', 'Gionee', 'Coolpad', 'General Mobile', 'Digma',
       'Yota Devices', 'Gome', 'Haier', 'Egreat', 'Kata', 'Mito',
       'Poptel', 'Flylion', 'AGM', 'SenseIT', 'DEXP', 'Archos', 'Teclast',
       'Fly', 'MLS', 'Keecoo', 'InFocus', 'Highscreen', 'Panasonic',
       'Inoi', 'InnJoo', 'Iris', 'LeTV', 'Itoos', 'MTC', 'Black Fox',
       'Acer', 'Nuu', 'Celkon', 'Chuwi', 'Walton', 'Xiaolajiao',
       'Hisense', 'Lava', 'myPh

в device_brand помимо пропусков есть  "". заменим их на None

In [9]:
df_total["device_brand"] = df_total["device_brand"].replace("", None)

In [10]:
df_total["device_brand"].unique()

array(['Samsung', 'Realme', 'Apple', 'Huawei', 'Xiaomi', None, 'Vivo',
       'OPPO', '(not set)', 'OnePlus', 'Sony', 'BQ', 'ZTE', 'Asus',
       'Umidigi', 'Meizu', 'itel', 'LG', 'LeEco', 'Nokia', 'Blackview',
       'Google', 'Lenovo', 'BlackBerry', 'Tecno', 'Cubot', 'Alcatel',
       'Neffos', 'Vsmart', 'Motorola', 'Infinix', 'TCL', 'Vertex',
       'Philips', 'Oukitel', 'Vernee', 'Micromax', 'HOMTOM', 'CAT',
       'Prestigio', 'Ulefone', 'DOOGEE', 'Leagoo', 'TP-Link',
       'SonyEricsson', 'HTC', 'Mozilla', 'Karbonn', 'Sharp', 'Wileyfox',
       'POCO', 'A1', 'Gionee', 'Coolpad', 'General Mobile', 'Digma',
       'Yota Devices', 'Gome', 'Haier', 'Egreat', 'Kata', 'Mito',
       'Poptel', 'Flylion', 'AGM', 'SenseIT', 'DEXP', 'Archos', 'Teclast',
       'Fly', 'MLS', 'Keecoo', 'InFocus', 'Highscreen', 'Panasonic',
       'Inoi', 'InnJoo', 'Iris', 'LeTV', 'Itoos', 'MTC', 'Black Fox',
       'Acer', 'Nuu', 'Celkon', 'Chuwi', 'Walton', 'Xiaolajiao',
       'Hisense', 'Lava', 'myPhone'

In [11]:
l_apple_pass = df_total[(df_total["device_brand"].isnull())&((df_total["device_browser"]=="Safari")|(df_total["device_os"]=="Macintosh")
                                                             |(df_total["device_browser"]=="Safari (in-app)")|(df_total["device_os"]=="iOS"))].index.tolist()
len(l_apple_pass)

31556

In [12]:
df_total.loc[df_total.index.isin(l_apple_pass), "device_brand"] = df_total.loc[df_total.index.isin(l_apple_pass), "device_brand"].fillna("Apple")

In [13]:
df_total.loc[df_total.index.isin(l_apple_pass), "device_brand"]

10         Apple
11         Apple
103        Apple
169        Apple
270        Apple
           ...  
1320497    Apple
1320592    Apple
1320625    Apple
1320677    Apple
1320686    Apple
Name: device_brand, Length: 31556, dtype: object

так же попробуем найти samsung

In [14]:
l_samsung_pass = df_total[(df_total["device_brand"].isnull())&((df_total["device_browser"]=="Samsung Internet")
                                                               |(df_total["device_browser"]=="Edge")|(df_total["device_os"]=="Samsung"))].index.tolist()
len(l_samsung_pass)

11167

In [15]:
df_total.loc[df_total.index.isin(l_samsung_pass), "device_brand"] = df_total.loc[df_total.index.isin(l_samsung_pass), "device_brand"].fillna("Samsung")

In [16]:
df_total["device_brand"].isna().sum()

np.int64(198430)

In [17]:
df_total["device_brand"].value_counts()

device_brand
Apple       403699
Samsung     259036
Xiaomi      214883
Huawei      143053
Realme       14771
             ...  
Kenxinda         1
Tagital          1
Dynamic          1
FREETEL          1
Maxvi            1
Name: count, Length: 200, dtype: int64

In [18]:
df_total["device_browser"].value_counts()

device_browser
Chrome                              735917
Safari                              313986
YaBrowser                            88304
Safari (in-app)                      59115
Android Webview                      45546
Samsung Internet                     34918
Opera                                18690
Firefox                              11460
Edge                                 11016
UC Browser                             584
Mozilla Compatible Agent               321
Android Runtime                        238
[FBAN                                  112
Instagram 208.0.0.32.135 Android        70
MRCHROME                                62
Instagram 209.0.0.21.119 Android        49
Android Browser                         46
Puffin                                  34
Internet Explorer                       24
Opera Mini                              22
Android                                 19
Instagram 202.0.0.23.119                17
Maxthon                                

In [19]:
df_total[df_total["device_brand"].isnull()]["device_browser"].value_counts()

device_browser
Chrome                              137587
YaBrowser                            37254
Opera                                12856
Firefox                              10008
Mozilla Compatible Agent               310
[FBAN                                  110
Instagram 208.0.0.32.135 Android        70
MRCHROME                                62
Instagram 209.0.0.21.119 Android        49
Android                                 19
Internet Explorer                       16
Instagram 194.0.0.36.172 Android        14
Instagram 202.0.0.37.123 Android        12
(not set)                               10
Maxthon                                  7
Puffin                                   6
Instagram 212.0.0.38.119 Android         6
helloworld                               6
Instagram 213.0.0.29.120 Android         4
Instagram 216.1.0.21.137 Android         4
Instagram 192.0.0.35.123 Android         3
SeaMonkey                                3
Mozilla                                

In [20]:
df_total_new = df_total.drop(columns = ["session_id_ch", "device_os"])

In [21]:
for column in df_total_new.columns.tolist():
    print(column, df_total_new[column].value_counts(normalize=True).head(10))

utm_medium utm_medium
banner             0.351606
cpc                0.191358
cpm                0.153181
(none)             0.145374
referral           0.063948
organic            0.024577
push               0.019048
email              0.016580
stories            0.007450
blogger_channel    0.004791
Name: proportion, dtype: float64
device_category device_category
mobile     0.808040
desktop    0.182501
tablet     0.009459
Name: proportion, dtype: float64
device_brand device_brand
Apple        0.359713
Samsung      0.230812
Xiaomi       0.191470
Huawei       0.127466
Realme       0.013162
(not set)    0.011797
OPPO         0.009308
Vivo         0.008113
OnePlus      0.005778
Asus         0.005194
Name: proportion, dtype: float64
device_browser device_browser
Chrome              0.557213
Safari              0.237740
YaBrowser           0.066861
Safari (in-app)     0.044760
Android Webview     0.034486
Samsung Internet    0.026439
Opera               0.014151
Firefox             0.008677

In [22]:
for column in df_total_new.drop(columns = "target").columns.tolist():
    l_column = df_total_new[column].value_counts(normalize = True)[df_total_new[column].value_counts(normalize = True)<0.01].index.tolist()
    df_total_new.loc[df_total_new[column].isin(l_column), column]="Other"

In [17]:
for column in df_total_new.columns.tolist():
    print(f" для {column} значения {df_total_new[column].unique().tolist()}")

 для utm_medium значения ['cpm', '(none)', 'banner', 'cpc', 'email', 'organic', 'Other', 'referral', 'push']
 для device_category значения ['mobile', 'desktop', 'Other']
 для device_brand значения ['Samsung', 'Realme', 'Apple', 'Huawei', 'Xiaomi', nan, 'Other', '(not set)']
 для device_browser значения ['Chrome', 'Samsung Internet', 'Safari', 'YaBrowser', 'Other', 'Android Webview', 'Safari (in-app)', 'Opera']
 для geo_country значения ['Russia', 'Other']
 для geo_city значения ['Other', 'Moscow', 'Novosibirsk', 'Saint Petersburg', 'Krasnoyarsk', '(not set)', 'Samara', 'Yekaterinburg', 'Nizhny Novgorod', 'Krasnodar', 'Kazan', 'Ufa']
 для utm_campaign значения ['FTjNLDyTrXaWYgZymFkV', 'LTuZkdKfxRGVceoWkVyg', 'gecBYcKZCPMcVYdSSzKP', 'LEoPHuyFvzoNfnzGgfcd', nan, 'XHNUiSKKGTEpSAlaVMFQ', 'Other', 'bxOTvPtyGSdUrbwoXCPO', 'okTXSMadDkjvntEHzIjp', 'SgIUDYUKnyWHVowUOqid', 'TmThBvoCcwkCZZUWACYq', 'LwJZxKuWmvOhPsaCeRjG', 'sbJRYgVfvcnqKJNDDYIr']
 для utm_source значения ['MvfHsxITijuriZxsqZqt', 'fD

In [23]:
for column in df_total_new.columns.tolist():
    print(column, df_total_new[column].value_counts(normalize=True).head(10))

utm_medium utm_medium
banner      0.351606
cpc         0.191358
cpm         0.153181
(none)      0.145374
referral    0.063948
Other       0.034329
organic     0.024577
push        0.019048
email       0.016580
Name: proportion, dtype: float64
device_category device_category
mobile     0.808040
desktop    0.182501
Other      0.009459
Name: proportion, dtype: float64
device_brand device_brand
Apple        0.359713
Samsung      0.230812
Xiaomi       0.191470
Huawei       0.127466
Other        0.065580
Realme       0.013162
(not set)    0.011797
Name: proportion, dtype: float64
device_browser device_browser
Chrome              0.557213
Safari              0.237740
YaBrowser           0.066861
Safari (in-app)     0.044760
Android Webview     0.034486
Samsung Internet    0.026439
Other               0.018349
Opera               0.014151
Name: proportion, dtype: float64
geo_country geo_country
Russia    0.969827
Other     0.030173
Name: proportion, dtype: float64
geo_city geo_city
Moscow    

In [24]:
df_total_new.head()

,utm_medium,device_category,device_brand,device_browser,geo_country,geo_city,utm_campaign,utm_source,utm_adcontent,target
0,cpm,mobile,Samsung,Chrome,Russia,Other,FTjNLDyTrXaWYgZymFkV,MvfHsxITijuriZxsqZqt,PkybGvWbaqORmxjNunqZ,0
1,(none),mobile,Samsung,Samsung Internet,Russia,Other,LTuZkdKfxRGVceoWkVyg,fDLlAcSmythWSCVMvqvL,JNHcPlZPxEMWDnRiyoBf,0
2,banner,mobile,Samsung,Chrome,Russia,Other,gecBYcKZCPMcVYdSSzKP,ZpYIoDJMcFzVoPFsHGJL,JNHcPlZPxEMWDnRiyoBf,0
3,banner,mobile,Realme,Chrome,Russia,Other,LEoPHuyFvzoNfnzGgfcd,ZpYIoDJMcFzVoPFsHGJL,JNHcPlZPxEMWDnRiyoBf,0
4,(none),mobile,Apple,Safari,Russia,Moscow,LTuZkdKfxRGVceoWkVyg,fDLlAcSmythWSCVMvqvL,JNHcPlZPxEMWDnRiyoBf,0


In [25]:
df_total_new.to_csv("total.csv", index=False)

In [2]:
df_total_new = pd.read_csv("total.csv")

In [3]:
df_total_new.columns.tolist()

['utm_medium',
 'device_category',
 'device_brand',
 'device_browser',
 'geo_country',
 'geo_city',
 'utm_campaign',
 'utm_source',
 'utm_adcontent',
 'target']

In [5]:
train, test = train_test_split(df_total_new, test_size=0.5, stratify=df_total_new["target"], shuffle=True, random_state=42)

In [6]:
X = train.drop(columns = "target")
y = train["target"]

In [7]:
X.shape, y.shape

((660355, 9), (660355,))

In [8]:
train.shape, test.shape

((660355, 10), (660355, 10))

In [10]:
pipeline = Pipeline(steps = [
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OrdinalEncoder()),
    ("model", LogisticRegression())])

cv = cross_val_score(pipeline, X, y, cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42), scoring = "roc_auc", verbose = 2)
print(cv.mean(), cv.std())

[CV] END .................................................... total time=  10.0s
[CV] END .................................................... total time=  10.1s
[CV] END .................................................... total time=  10.0s
[CV] END .................................................... total time=   9.9s
[CV] END .................................................... total time=  10.1s
0.6105973906674379 0.003391342933743517


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   51.0s finished


In [11]:
pipeline_rf = Pipeline(steps = [
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OrdinalEncoder()),
    ("model", RandomForestClassifier())])

cv = cross_val_score(pipeline_rf, X, y, cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42), scoring = "roc_auc", verbose = 2)
print(cv.mean(), cv.std())

[CV] END .................................................... total time= 2.1min
[CV] END .................................................... total time= 2.1min
[CV] END .................................................... total time= 2.2min
[CV] END .................................................... total time= 2.2min
[CV] END .................................................... total time= 2.2min
0.6624432857138478 0.005295002784173997


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 10.9min finished


In [13]:
pipeline_lgb = Pipeline(steps = [
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OrdinalEncoder()),
    ("model", LGBMClassifier(force_col_wise=True, verbose = 0))])

cv = cross_val_score(pipeline_lgb, X, y, cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42), scoring = "roc_auc", verbose = 2)
print(cv.mean(), cv.std())

C:\Users\vasan\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[CV] END .................................................... total time=  10.4s


C:\Users\vasan\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[CV] END .................................................... total time=  10.1s


C:\Users\vasan\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


...

C:\Users\vasan\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[CV] END .................................................... total time=  10.0s


C:\Users\vasan\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[CV] END .................................................... total time=   9.9s
0.6908778258469245 0.002663382249117309


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   51.2s finished


In [15]:
pipeline_hist = Pipeline(steps = [
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OrdinalEncoder()),
    ("model", HistGradientBoostingClassifier())])

cv = cross_val_score(pipeline_hist, X, y, cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42), scoring = "roc_auc", verbose = 2)
print(cv.mean(), cv.std())

[CV] END .................................................... total time=  10.8s
[CV] END .................................................... total time=  12.3s
[CV] END .................................................... total time=  13.6s
[CV] END .................................................... total time=  12.2s
[CV] END .................................................... total time=  13.8s
0.6907668242977679 0.0024759666066704426


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.1min finished


In [16]:
pipeline_cat = Pipeline(steps = [
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OrdinalEncoder()),
    ("model", CatBoostClassifier(early_stopping_rounds=100, iterations=1000,verbose = 0))])

cv = cross_val_score(pipeline_cat, X, y, cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42), scoring = "roc_auc", verbose = 2)
print(cv.mean(), cv.std())

[CV] END .................................................... total time= 1.5min
[CV] END .................................................... total time= 1.4min
[CV] END .................................................... total time= 1.5min
[CV] END .................................................... total time= 1.5min
[CV] END .................................................... total time= 1.5min
0.6846654824417756 0.003482560752360747


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  7.3min finished


In [17]:
X_total = df_total_new.drop(columns = "target")
y_total = df_total_new["target"]

In [18]:
cv_lgb = cross_val_score(pipeline_lgb, X_total, y_total, cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42), scoring = "roc_auc", verbose = 2)
print(cv_lgb.mean(), cv_lgb.std())

C:\Users\vasan\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[CV] END .................................................... total time=  20.8s


C:\Users\vasan\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[CV] END .................................................... total time=  20.3s


C:\Users\vasan\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


...

C:\Users\vasan\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[CV] END .................................................... total time=  20.4s


C:\Users\vasan\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[CV] END .................................................... total time=  20.3s
0.6922807306055215 0.0014590555319827412


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.7min finished


In [19]:
cv_hist = cross_val_score(pipeline_hist, X_total, y_total, cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42), scoring = "roc_auc", verbose = 2)
print(cv_hist.mean(), cv_hist.std())

[CV] END .................................................... total time=  28.7s
[CV] END .................................................... total time=  28.3s
[CV] END .................................................... total time=  27.2s
[CV] END .................................................... total time=  28.3s
[CV] END .................................................... total time=  28.1s
0.6918888141355239 0.0015018478648652816


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  2.4min finished


In [20]:
x1_train, x1_test, y1_train, y1_test = train_test_split(X_total, y_total, test_size= 0.2, shuffle=True, random_state=42)

In [24]:
model_hist = pipeline_hist.fit(x1_train, y1_train)
pred_hist = model_hist.predict_proba(x1_test)[:,1]
model_hist_score = roc_auc_score(y1_test, pred_hist)
print(model_hist_score)

0.6893330494795329


In [25]:
model_lgb = pipeline_lgb.fit(x1_train, y1_train)
pred_lgb = model_lgb.predict_proba(x1_test)[:,1]
model_lgb_score = roc_auc_score(y1_test, pred_lgb)
print(model_lgb_score)

C:\Users\vasan\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


0.6892428199434046


In [26]:
model_total = pipeline_hist.fit(X_total, y_total)

In [28]:
joblib.dump({
    "model":model_total,
    "metadata":{
        "name" : "predict_model",
        "author" : "Andrey Vasilkov",
        "version" : 1,
        "date" : datetime.datetime.now(),
        "roc_auc" : model_hist_score
    }
}, "model.pkl")
        

['model.pkl']

In [34]:
df_random_choice = df.loc[np.random.randint(0,1860043),:].to_json()
print(df_random_choice)

{"session_id":"4221571881407069139.1628158934.1628158934","client_id":"982911298.1628158931","visit_date":1628121600000,"visit_time":"13:00:00","visit_number":1,"utm_source":"ZpYIoDJMcFzVoPFsHGJL","utm_medium":"banner","utm_campaign":"LEoPHuyFvzoNfnzGgfcd","utm_adcontent":"JNHcPlZPxEMWDnRiyoBf","utm_keyword":null,"device_category":"mobile","device_os":null,"device_brand":"Apple","device_model":null,"device_screen_resolution":"320x568","device_browser":"Safari","geo_country":"Russia","geo_city":"Arkhangelsk"}
